# Merging data frames

Merging in Pandas is similar to joining tables with SQL. If you are a SQL user then you will feel right at home.  

The basic idea is to combine the attributes of several dataframes based on a common value.  In this case we are adding new columns to an existing dataframe where with the concat method we were adding new rows.

Lets import some data

In [ ]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import numpy as np

eagle = gpd.read_file("data/BAEA_Nests.shp")
eagle_sources = pd.read_csv("data/BAEA_sources.csv")
eagle_surveys = pd.read_csv("data/BAEA_surveys.csv")
eagle.head()

The eagle_sources data frame contains 4 rows and is an example of a lookup table.  These were more common in the past when storage was more expensive and it was possible to save considerable space in a database by storing coded values and then linking the codes to text values with a lookup table. In this case the code could be stored as a single byte, where as the text string would require at least 30 bytes.  This level of savings may not be that important with the amount of storage available on modern computers but if you have large datasets it can still be significant.

In this example there are many coded values in the eagle GeoDataFrame for every entry in the eagle_sources data frame.  A many to one relationship

In [ ]:
eagle_sources

The eagle_surveys table is an example where there are many entries for each entry in the eagle GeoDataFrame. A one to many relationship. In this case each record in the eagle_sources data frame represents the results of an eagle nest surveys.  Surveys were conducted every week during the eagle breeding season so there are 1997 survey records covering 67 eagle nests.

In [ ]:
eagle_surveys

Lets add a code field to the eagles data and populate it with a random number from 1 to 4.

In [ ]:
eagle['code']=np.random.randint(1,5,size=67)
eagle.sort_values('nest_id')

Now lets merge the eagle table with the eagle_source table into a new table called eagle_merge.  The merge command is called on the Pandas object itself.  It takes two data frames as parameters, the first is considered the left data frame and the second is considered the right data frame.  This will be important later on.  

The how parameter specifies the type of join, possibliities include inner, left, right, and outer which will be familiar to SQL users. We'll discuss this later on.  

The on parameter specifies the field name to join on and can be used when the field name is the same in both tables.  

In [ ]:
eagle_merge = pd.merge(eagle, eagle_sources, how='inner', on='code').sort_values('nest_id')
eagle_merge

The source field can now be used like any other field in the table, for instance as the index for a pivot table, but it doesn't take up any more space on disk.

In [ ]:
pd.pivot_table(eagle_merge, index=['source', 'status'], values='nest_id', aggfunc='count')

Our new dataset can also be merged again with the eagle_survey data frame. This results in a one to many relationship.  **NOTE** In this example the columns we are joining on have different names so we cannot use the *on* parameter, instead we have to specify the column names for each data frame using the *left_on* and *right_on* parameters.

In [ ]:
eagle_merge2 = pd.merge(eagle_merge, eagle_surveys, how='inner', left_on='nest_id', right_on='nest').sort_values(['nest_id', 'date'])
eagle_merge2

Notice that in this case we end up with 1766 rows even though there are 1997 rows in the eagle_surveys table.  This is because we are doing an "inner join" which means that the resulting dataframe only has a record for cases where there is a match between the left_on and right_on columns.  Another possibility is a "left join" which means that there is at least one record for every row in the left table whether or not there is a corresponding record in the right table.  If there is not a corresponding record in the right table then the field values for the right table will be NaN.

In [ ]:
eagle_merge2 = pd.merge(eagle_merge, eagle_surveys, how='left', left_on='nest_id', right_on='nest').sort_values(['nest_id', 'date'])
eagle_merge2

In this case we have the same number of records as we did with an inner join which tells us that there are no records in the left table that have no corresponding record in the right table.

With a "right join" there will be a record for every record in the right table whether or not there is a corresponding record in the left table.  

In [ ]:
eagle_merge2 = pd.merge(eagle_merge, eagle_surveys, how='right', left_on='nest_id', right_on='nest').sort_values(['nest_id', 'date'])
eagle_merge2

We can identify the surveys that have no corresponding record in the eagle table by summarizing the resulting dataframe by nest and getting a count of the nest_id field. 

In [ ]:
eagle_merge2.groupby(by='nest').count()['nest_id'].sort_values()

As with concatenation, Pandas has a rich ecosystem for merging and joining data and there is a lot of additional functionality that canbe achieved with the merge method as well as a good bit of overlap with other methods.  Again I would refer you to the documentation for specifics on each method and in particular the page in the Pandas user guide on [Merge, Join, Concatenate, etc](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) for more information